In [17]:
import utils
import pathlib
utils.mount_src()

from data_loader import DataLoader

CONFIG = utils.load_config()
DATA_PATH = CONFIG["images_path"]
MODEL_PATH = pathlib.Path("../models")
EPOCHS = 30
INPUT_SHAPE = (224, 224, 3)

In [19]:
from tensorflow.keras.models import Model
from tensorflow.keras import layers
import tensorflow as tf

def create_dummy():
    # Define input shape (224x224x3 for images)
    input_shape = (224, 224, 3)

    # Dummy architecture: just a single dense layer after flattening
    inputs = Input(shape=input_shape)
    x = layers.Rescaling(1./255)(inputs)
    x = Dense(128, activation='relu')(inputs)  # Flatten step is skipped

    output_year = tf.keras.layers.Dense(1, name="year", activation="sigmoid")(x)
    output_lat = tf.keras.layers.Dense(1, name="lat", activation="sigmoid")(x)
    output_lon = tf.keras.layers.Dense(1, name="lon", activation="sigmoid")(x)

    return tf.keras.Model(inputs=inputs, outputs={"year": output_year, "lat": output_lat, "lon": output_lon})

dummy = create_dummy()

# Compile the model
dummy.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Save the dummy model
dummy.save(MODEL_PATH / "dummy_model.keras")

In [20]:
from scaler import Scaler
import numpy as np

def generate_pred_for_image(model: tf.keras.Model, img_path: str, target_size=(224, 224)) -> dict[str, np.ndarray]:
    img = tf.io.read_file(img_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, target_size)
    img = tf.expand_dims(img, axis=0)

    pred = model.predict(img)
    return Scaler.descale_preds(pred)

In [ ]:
from tensorflow.keras.models import load_model

model = load_model("../models/dummy_model.keras")
image_path = "C:\\Users\\guisf\\Downloads\\test.JPG"

pred = generate_pred_for_image(model, image_path)
pred["year"][0][0]
